$\Huge{Exercise \ \ 1}$

Consider the following optimization problem:
$$
\begin{aligned}
\max 0.65 x_{1}+0.75 x_{2}-5.85 x_{3}-8.35 x_{4}-12.38 x_{5}+30 x_{6} & \\
\text { s.t. }-0.5 x_{1}-3.8 x_{3}-0.9 x_{4}+0.9 x_{5} & \leq 24.8 \\
x_{1}-x_{4}-x_{6} & \leq 0.8 \\
-2.9 x_{2}-1.9 x_{3}-0.9 x_{6} & \leq-71.4 \\
4.7 x_{4}+8.02 x_{5}+6.15 x_{6} & \leq 95 \\
-0.7 x_{1}-1.4 x_{2}+7.9 x_{3}-2.9 x_{4}+0.9 x_{5} & \leq-47 \\
x_{i} \geq 0 \forall i \in\{1,2, \ldots, 6\}
\end{aligned}
$$

In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 29.3 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 


In [ ]:
from pyomo.environ import * 

In [ ]:
import numpy as np

In [ ]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

In [ ]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])


(6, 7)
Num rows: 6
Num cols: 7


In [ ]:
print(coef)

[[  0.65   0.75  -5.85  -8.35 -12.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [ ]:
# create a model
model_lab4 = ConcreteModel()

In [ ]:
#creating decision variable
N = coef.shape[1]-1

In [ ]:
#creating no of constraints
M = coef.shape[0]-1 

In [ ]:
#taking objective coefficients from .txt file
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(6,)
[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [ ]:
# taking constraint coefficient from .txt file
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [ ]:
# taking rhs of constraint coefficient from .txt file
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [ ]:
#Since the variables have the same upper bound and lower bound, we can simply use scalars to store them
lower_bound = 0
upper_bound = np.inf

In [ ]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [ ]:
#declare the decision variables in the model
model_lab4.x = Var(col_indices)

In [ ]:
# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [ ]:
# create a ConstraintList to hold multiple constraints
model_lab4.constraints = ConstraintList()

In [ ]:
for i in row_indices:
  model_lab4.constraints.add(summation(constr_coef[i],model_lab4.x) <= constr_rhs[i])

In [ ]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab4.x[j].setlb(lower_bound)
  model_lab4.x[j].setub(upper_bound)

In [ ]:
# add the model objective
model_lab4.objective = Objective(expr = summation(obj_coef,model_lab4.x), sense=maximize)

In [ ]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [ ]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


$ \Huge{solution \ \ \ of \ \ \ 1.7}$

Solver status is warning
and Solver termination condition is  unbounded 

we get unbounded solution because of some of the variable has no such constraints that bound variable or objective sense is wrong
 
 because of that it take value as $\infty \ \  $ 

$\Huge{solution \ \ of \ \ 1.8}$

first i changed the objective sense but it does not  gives optimal solution 

now i change variable upper bound and i got objective function is unbounded because of $ x_2 $ takes value as $\infty$ 

by changing many upper bound of $x_2$ i got if $ x_2 <20.1032727 $ solution is infeasible if $ x_2 >20.1032727$ then we got optimal solution 

since minimum upper bound of $x_2$ is 20.1032727 so i choose it for upper bound

In [ ]:
model_lab4.x[1].setub(20.1032727)

In [ ]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model_lab4.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab4.x[i].value)

print('\nConstraints')
model_lab4.constraints.display()


Objective =  -180.37644654644697

Decision Variables
x[ 0 ] =  21.012765
x[ 1 ] =  20.103273
x[ 2 ] =  6.8950029
x[ 3 ] =  20.212761
x[ 4 ] =  0.0
x[ 5 ] =  4.0922851e-06

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :       -54.89887842 :  24.8
      2 :  None : 0.7999999077149013 :   0.8
      3 :  None :  -71.4000008930566 : -71.4
      4 :  None :  95.00000186755337 :  95.0
      5 :  None :       -47.00000169 : -47.0


$ \Huge{solution  \ \ of \ \  1.9} $

first i changed the objective sense but it does not  gives optimal solution 

now i change variable upper bound and i got objective function is unbounded because of $ x_2 $ takes value as $\infty$ 

by changing many upper bound of $x_2$ i got if $ x_2 <20.1032727 $ solution is infeasible if $ x_2 >20.1032727$ then we got optimal solution 

since minimum upper bound of $x_2$ is 20.1032727 so i choose it for upper bound

$ \huge{optimal \ \ objective \ \ function \ \ value =-180.37644654644697}$


$\huge{value \ \ of \ \ variable}$

$x_1$ =  21.012765

$x_2$ =  20.103273


$x_3$ =  6.8950029

$x_4$ =  20.212761

$x_5$ =  0.0

$x_6$ =  4.0922851e-06

constraint 1 is inactive 

constraint 2,3,4,5 is active